<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2021/blob/main/NetworkFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from scipy.optimize import newton, root

In [3]:
g=9.81
rho=1e3
mu=1e-3
D=0.1
eD=1e-3

In [8]:
# Calculates the friction factor using Churchill correlation 
def f(Re,eD):
    A=(2.457*np.log(1/((7./Re)**0.9+0.27*eD)))**16
    B=(37530/Re)**16
    return 2*((8/Re)**12 + 1/(A+B)**(1.5))**(1/12)

In [18]:
# Given a flow Q, calculates delta Head  (head at ToNode) - (head at FromNode)
def dHead(Q,L,K,D,eD):
    v=np.abs(Q/(np.pi*D**2/4))  #ensures that we do not end up with negative Reynolds numbers
    Re= rho*v*D/mu
    return -np.sign(Q)*(K+4*f(Re,eD)*L/D)*(v**2/2)/g  #incorporate back in sign (directionality) of Q (flow)

In [19]:
# Given deltaHead, it calculates the flow (with proper signage)
def flow(deltaHead, L, K, D, eD):
    dH_abs=np.abs(deltaHead)  #ensures no negative Re
    if dH_abs<1e-10: #takes care of situation where deltaHead is extremely small, for which
    # the numerical solver (newton) has difficulty converging
        return 0.0

    Qguess=np.pi*D**2/4*np.sqrt(dH_abs/(K+4*0.01*L/D)*2*g)  #Initial guess for flow rate, assuming a reasonable
    #guess of 0.01 for the Fanning friction factor

    #function to zero - Calculated deltahead based on the iterated flow rate (Q) must be equal to the deltaHead
    #given.
    def f_to_zero(Q):
        return dHead(Q,L,K,D,eD)-dH_abs

    resulting_flow_rate=newton(f_to_zero,Qguess)

    return np.sign(deltaHead)*resulting_flow_rate  #Correct for directionality of deltaHead

In [20]:
#fromNode, toNode, L, K
pipes = np.array([[0, 1, 1.5, 1.5],
                  [0, 2, 2., 2.],
                  [1,2, 1.5, 0],
                  [1, 3, 1.5, 0],
                  [2,4, 0.5, 0],
                  [3,4, 3., 2]])

z=np.array([30.,0,0,0,0 ])
externalP=np.array([2e5, 0.])
externalNodes=[0, 4]
fromNodes=(pipes[:,0]).astype(np.int32)
toNodes=(pipes[:,1]).astype(np.int32)
lengths=pipes[:,2]
K = pipes[:,3]

nPipes = pipes.shape[0]
pipeD = np.full(nPipes, D)
pipe_eD = np.full(nPipes,eD)

# number of nodes will the maximum in the column for ToNodes because numbering of pipes is always
# from nodes of lower index to a node of higher index.  Add 1, because indexing starts at 0.
nNodes = (np.max(pipes[:,1])).astype(np.int32)+1
heads=np.zeros(nNodes)
heads[externalNodes]=externalP/rho/g + z[externalNodes]

In [21]:
internalNodes=list(set(range(nNodes))-set(externalNodes))

In [22]:
# from iterated values of heads, calculate flows through each pipe
def calc_flows(heads):
    flows=np.zeros(nPipes)
    for i in range(nPipes):
        deltaHead = heads[toNodes[i]] - heads[fromNodes[i]]
        flows[i]=flow(deltaHead, lengths[i], K[i], pipeD[i], pipe_eD[i])
    return flows

# from flows through each pipe, calculates net flows for all nodes (internal and external)
def calc_netQ(flows):
    netQ=np.zeros(nNodes)
    np.add.at(netQ,toNodes,flows)
    np.add.at(netQ,fromNodes,-flows)  # flows leaving a node are negative
    return netQ

In [23]:
# function to find the roots (zeros) of.  The net flow for all internal nodes must be zero
# achieved by iterating through the unknown internal node pressures
def netQ_internal(internalP):
    heads[internalNodes]=internalP/rho/g + z[internalNodes]
    return calc_netQ(calc_flows(heads))[internalNodes]


In [24]:
# to obtain initial guess for internal pressures, reasonable to say that the internal heads are the average
# of the known external heads.
internalHeadGuess=np.mean(heads[externalNodes])

# H = P/(rho*g) + z : calculate internal P guess from internal H guesses
internalPGuess = (internalHeadGuess - z[internalNodes])*rho*g 

In [25]:
# solve for unknown internal node pressures
result=root(netQ_internal, internalPGuess)
result.x

array([82620.70449187, 53908.25549997, 74034.72098688])

In [26]:
# flows through each pipe
calculated_flows = calc_flows(heads)
for i in range(nPipes):
    print(f'from: {int(pipes[i,0])}    to: {int(pipes[i,1])}    flow: {calculated_flows[i]}')

from: 0    to: 1    flow: 0.1680965483810236
from: 0    to: 2    flow: 0.1505538615298285
from: 1    to: 2    flow: 0.10887171792125305
from: 1    to: 3    flow: 0.05922483045544484
from: 2    to: 4    flow: 0.25942557945057937
from: 3    to: 4    flow: 0.05922483046009628


In [27]:
# net flow, pressure, and elevation for each node
calculated_netQ = calc_netQ(calc_flows(heads))

pressure = np.zeros(nNodes)
pressure[externalNodes]= externalP
pressure[internalNodes]= result.x
for i in range(nNodes):
    print(f'Node {i}   Pressure: {pressure[i]}   z: {z[i]}    NetQ: {calculated_netQ[i]}')

Node 0   Pressure: 200000.0   z: 30.0    NetQ: -0.31865040991085214
Node 1   Pressure: 82620.70449186761   z: 0.0    NetQ: 4.325720337483574e-12
Node 2   Pressure: 53908.255499966406   z: 0.0    NetQ: 5.022093851891896e-13
Node 3   Pressure: 74034.72098687997   z: 0.0    NetQ: -4.651438956226883e-12
Node 4   Pressure: 0.0   z: 0.0    NetQ: 0.31865040991067567
